# E-Commerce Business Analytics Dashboard

## Executive Summary

This notebook provides comprehensive analysis of e-commerce business performance, focusing on revenue trends, customer satisfaction, product performance, and operational metrics. The analysis framework is designed to be configurable for different time periods and can be easily adapted for ongoing business monitoring.

### Key Business Questions Addressed:
- How is revenue performing compared to previous periods?
- What are the monthly growth trends?
- Which product categories drive the most revenue?
- How does geographic distribution affect sales performance?
- What is the relationship between delivery speed and customer satisfaction?
- What are our key operational performance indicators?

---

## Table of Contents

1. [Configuration & Setup](#configuration--setup)
2. [Data Dictionary](#data-dictionary)
3. [Data Loading & Validation](#data-loading--validation)
4. [Revenue Performance Analysis](#revenue-performance-analysis)
5. [Product Category Analysis](#product-category-analysis)
6. [Geographic Performance Analysis](#geographic-performance-analysis)
7. [Customer Experience Analysis](#customer-experience-analysis)
8. [Executive Summary & Key Insights](#executive-summary--key-insights)

---

## Configuration & Setup

Configure the analysis parameters and import required modules.

In [ ]:
# Analysis Configuration
TARGET_YEAR = 2023
COMPARISON_YEAR = 2022
TARGET_MONTH = None  # Set to specific month (1-12) or None for full year
DATA_PATH = "ecommerce_data"

# Display Settings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_white"

# Import custom modules
from data_loader import EcommerceDataLoader
from business_metrics import EcommerceMetrics

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)

print(f"Analysis Configuration:")
print(f"Target Year: {TARGET_YEAR}")
print(f"Comparison Year: {COMPARISON_YEAR}")
print(f"Target Month: {TARGET_MONTH if TARGET_MONTH else 'Full Year'}")
print(f"Data Path: {DATA_PATH}")

## Data Dictionary

### Key Business Terms and Metrics

| Metric | Definition | Business Relevance |
|--------|------------|-------------------|
| **Revenue** | Total monetary value from delivered orders | Primary business performance indicator |
| **Average Order Value (AOV)** | Total revenue / Number of orders | Customer spending behavior |
| **Month-over-Month Growth** | (Current Month - Previous Month) / Previous Month | Short-term trend analysis |
| **Customer Satisfaction Rate** | Percentage of orders with 4+ star reviews | Quality of customer experience |
| **Net Promoter Score (NPS)** | Promoters % - Detractors % | Customer loyalty indicator |
| **Delivery Speed** | Days between order purchase and customer delivery | Operational efficiency |
| **Fast Delivery Rate** | Percentage of orders delivered within 3 days | Premium service performance |

### Dataset Schema

**Orders Dataset:**
- `order_id`: Unique identifier for each order
- `customer_id`: Unique identifier for each customer
- `order_status`: Current status (delivered, shipped, canceled, etc.)
- `order_purchase_timestamp`: When the order was placed
- `order_delivered_customer_date`: When order was delivered to customer

**Order Items Dataset:**
- `order_id`: Links to orders dataset
- `product_id`: Unique identifier for each product
- `price`: Item price in USD
- `freight_value`: Shipping cost

**Products Dataset:**
- `product_id`: Links to order items
- `product_category_name`: Product category classification

**Customers Dataset:**
- `customer_id`: Links to orders dataset
- `customer_state`: Customer's state for geographic analysis

**Reviews Dataset:**
- `order_id`: Links to orders dataset
- `review_score`: Customer rating (1-5 stars)

---

## Data Loading & Validation

Load all datasets and perform basic validation checks.

In [ ]:
# Initialize data loader and load all datasets
loader = EcommerceDataLoader(DATA_PATH)
datasets = loader.load_all_datasets()

# Display dataset information
print("\nDataset Summary:")
dataset_info = loader.get_dataset_info()
info_df = pd.DataFrame(dataset_info).T
print(info_df)

In [ ]:
# Create analysis datasets
print(f"Preparing data for analysis...")

# Primary analysis dataset (target year)
sales_target = loader.create_sales_dataset(
    target_year=TARGET_YEAR, 
    target_month=TARGET_MONTH,
    order_status='delivered'
)

# Comparison dataset (previous year)
sales_comparison = loader.create_sales_dataset(
    target_year=COMPARISON_YEAR,
    target_month=TARGET_MONTH,
    order_status='delivered'
)

print(f"Target Period ({TARGET_YEAR}): {len(sales_target):,} records, {sales_target['order_id'].nunique():,} orders")
print(f"Comparison Period ({COMPARISON_YEAR}): {len(sales_comparison):,} records, {sales_comparison['order_id'].nunique():,} orders")

# Add enhanced datasets with additional information
sales_with_delivery = loader.add_delivery_metrics(sales_target)
sales_with_reviews = loader.get_review_data(sales_target)
sales_with_categories = loader.get_product_categories_data(sales_target)
sales_with_states = loader.get_customer_geographic_data(sales_target)

print(f"\nEnhanced datasets prepared successfully.")

## Revenue Performance Analysis

Comprehensive analysis of revenue trends, growth rates, and key performance indicators.

In [ ]:
# Initialize metrics calculator
metrics = EcommerceMetrics()

# Calculate revenue metrics with year-over-year comparison
revenue_metrics = metrics.calculate_revenue_metrics(sales_target, sales_comparison)

# Display key metrics
print("REVENUE PERFORMANCE METRICS")
print("=" * 40)
print(f"Total Revenue ({TARGET_YEAR}): ${revenue_metrics['total_revenue']:,.2f}")
print(f"Total Orders: {revenue_metrics['total_orders']:,}")
print(f"Average Order Value: ${revenue_metrics['average_order_value']:.2f}")
print(f"Average Item Price: ${revenue_metrics['average_item_price']:.2f}")
print(f"")
print("YEAR-OVER-YEAR COMPARISON")
print("=" * 40)
print(f"Revenue Growth: {revenue_metrics['revenue_growth_rate']:.2f}%")
print(f"Order Volume Growth: {revenue_metrics['order_growth_rate']:.2f}%")
print(f"AOV Growth: {revenue_metrics['aov_growth_rate']:.2f}%")

# Create revenue metrics summary table
comparison_data = {
    'Metric': ['Revenue', 'Orders', 'Average Order Value'],
    f'{TARGET_YEAR}': [f"${revenue_metrics['total_revenue']:,.0f}", 
                      f"{revenue_metrics['total_orders']:,}",
                      f"${revenue_metrics['average_order_value']:.2f}"],
    f'{COMPARISON_YEAR}': [f"${revenue_metrics['previous_revenue']:,.0f}",
                          f"{revenue_metrics['previous_orders']:,}",
                          f"${revenue_metrics['previous_aov']:.2f}"],
    'Growth Rate': [f"{revenue_metrics['revenue_growth_rate']:.2f}%",
                   f"{revenue_metrics['order_growth_rate']:.2f}%",
                   f"{revenue_metrics['aov_growth_rate']:.2f}%"]
}

comparison_df = pd.DataFrame(comparison_data)
print(f"\nDETAILED COMPARISON TABLE:")
print(comparison_df.to_string(index=False))

In [ ]:
# Calculate monthly trends
monthly_trends = metrics.calculate_monthly_trends(sales_target)

print(f"MONTHLY PERFORMANCE TRENDS ({TARGET_YEAR})")
print("=" * 50)
print(monthly_trends.to_string(index=False))

# Calculate average monthly growth
avg_monthly_growth = monthly_trends['revenue_growth'].mean()
print(f"\nAverage Monthly Growth Rate: {avg_monthly_growth:.2f}%")

In [ ]:
# Create revenue trend visualization
title_suffix = f"({TARGET_YEAR})"
revenue_plot = metrics.plot_revenue_trend(monthly_trends, title_suffix)
revenue_plot.show()

# Additional insights
print(f"\nKEY INSIGHTS:")
print(f"- Best performing month: Month {monthly_trends.loc[monthly_trends['revenue'].idxmax(), 'month']} with ${monthly_trends['revenue'].max():,.0f}")
print(f"- Worst performing month: Month {monthly_trends.loc[monthly_trends['revenue'].idxmin(), 'month']} with ${monthly_trends['revenue'].min():,.0f}")
print(f"- Revenue volatility (std dev): ${monthly_trends['revenue'].std():,.0f}")

## Product Category Analysis

Analysis of product category performance, revenue contribution, and market share.

In [ ]:
# Calculate product category performance
category_performance = metrics.calculate_product_performance(sales_with_categories)

print(f"PRODUCT CATEGORY PERFORMANCE ({TARGET_YEAR})")
print("=" * 60)
print(category_performance.to_string(index=False))

# Top performing categories
top_3_categories = category_performance.head(3)
print(f"\nTOP 3 REVENUE GENERATING CATEGORIES:")
for idx, row in top_3_categories.iterrows():
    print(f"1. {row['product_category_name']}: ${row['total_revenue']:,.0f} ({row['revenue_share']:.1f}% of total)")

In [ ]:
# Create category performance visualization
title_suffix = f"({TARGET_YEAR})"
category_plot = metrics.plot_category_performance(category_performance, title_suffix)
category_plot.show()

# Category insights
total_categories = len(category_performance)
top_3_share = top_3_categories['revenue_share'].sum()

print(f"\nCATEGORY INSIGHTS:")
print(f"- Total product categories: {total_categories}")
print(f"- Top 3 categories represent {top_3_share:.1f}% of total revenue")
print(f"- Most expensive category (avg price): {category_performance.loc[category_performance['avg_price'].idxmax(), 'product_category_name']} (${category_performance['avg_price'].max():.2f})")
print(f"- Highest volume category: {category_performance.loc[category_performance['total_items'].idxmax(), 'product_category_name']} ({category_performance['total_items'].max():,} items)")

## Geographic Performance Analysis

Regional sales performance and market penetration analysis.

In [ ]:
# Calculate geographic performance
geographic_performance = metrics.calculate_geographic_performance(sales_with_states)

print(f"GEOGRAPHIC PERFORMANCE BY STATE ({TARGET_YEAR})")
print("=" * 60)
print(geographic_performance.head(10).to_string(index=False))

# Top performing states
top_5_states = geographic_performance.head(5)
print(f"\nTOP 5 STATES BY REVENUE:")
for idx, row in top_5_states.iterrows():
    print(f"{idx+1}. {row['state']}: ${row['total_revenue']:,.0f} ({row['revenue_share']:.1f}% of total)")

In [ ]:
# Create geographic distribution map
title_suffix = f"({TARGET_YEAR})"
geo_plot = metrics.plot_geographic_distribution(geographic_performance, title_suffix)
geo_plot.show()

# Geographic insights
total_states = len(geographic_performance)
top_5_share = top_5_states['revenue_share'].sum()

print(f"\nGEOGRAPHIC INSIGHTS:")
print(f"- Active states: {total_states}")
print(f"- Top 5 states represent {top_5_share:.1f}% of total revenue")
print(f"- Highest revenue per customer: {geographic_performance.loc[geographic_performance['revenue_per_customer'].idxmax(), 'state']} (${geographic_performance['revenue_per_customer'].max():.2f})")
print(f"- Most loyal customers (orders/customer): {geographic_performance.loc[geographic_performance['orders_per_customer'].idxmax(), 'state']} ({geographic_performance['orders_per_customer'].max():.2f})")

## Customer Experience Analysis

Customer satisfaction metrics, delivery performance, and experience quality indicators.

In [ ]:
# Calculate customer satisfaction metrics
satisfaction_metrics = metrics.calculate_customer_satisfaction(sales_with_reviews)

print(f"CUSTOMER SATISFACTION METRICS ({TARGET_YEAR})")
print("=" * 50)
print(f"Average Rating: {satisfaction_metrics['average_rating']:.2f}/5.0")
print(f"Total Reviews: {satisfaction_metrics['total_reviews']:,}")
print(f"Satisfaction Rate (4+ stars): {satisfaction_metrics['satisfaction_rate']:.1f}%")
print(f"Net Promoter Score: {satisfaction_metrics['nps_score']:.1f}")

print(f"\nRATING DISTRIBUTION:")
rating_dist = satisfaction_metrics['rating_distribution']
for score in sorted(rating_dist.index):
    percentage = (rating_dist[score] / rating_dist.sum()) * 100
    print(f"{score} stars: {rating_dist[score]:,} reviews ({percentage:.1f}%)")

In [ ]:
# Calculate delivery performance metrics
delivery_metrics = metrics.calculate_delivery_performance(sales_with_delivery)

print(f"\nDELIVERY PERFORMANCE METRICS ({TARGET_YEAR})")
print("=" * 50)
print(f"Average Delivery Time: {delivery_metrics['average_delivery_days']:.1f} days")
print(f"Median Delivery Time: {delivery_metrics['median_delivery_days']:.1f} days")
print(f"Fast Delivery Rate (≤3 days): {delivery_metrics['fast_delivery_rate']:.1f}%")
print(f"Slow Delivery Rate (>7 days): {delivery_metrics['slow_delivery_rate']:.1f}%")

print(f"\nDELIVERY TIME DISTRIBUTION:")
delivery_dist = delivery_metrics['delivery_distribution']
for category in delivery_dist.index:
    percentage = (delivery_dist[category] / delivery_dist.sum()) * 100
    print(f"{category}: {delivery_dist[category]:,} orders ({percentage:.1f}%)")

In [ ]:
# Analyze satisfaction vs delivery speed relationship
combined_data = sales_with_delivery.merge(sales_with_reviews[['order_id', 'review_score']], on='order_id', how='inner')
satisfaction_delivery = metrics.analyze_satisfaction_vs_delivery(combined_data)

print(f"CUSTOMER SATISFACTION BY DELIVERY SPEED ({TARGET_YEAR})")
print("=" * 60)
print(satisfaction_delivery.to_string(index=False))

# Key insights
best_delivery = satisfaction_delivery.loc[satisfaction_delivery['avg_rating'].idxmax()]
worst_delivery = satisfaction_delivery.loc[satisfaction_delivery['avg_rating'].idxmin()]

print(f"\nDELIVERY INSIGHTS:")
print(f"- Best rated delivery category: {best_delivery['delivery_category']} ({best_delivery['avg_rating']:.3f} avg rating)")
print(f"- Worst rated delivery category: {worst_delivery['delivery_category']} ({worst_delivery['avg_rating']:.3f} avg rating)")
print(f"- Rating difference between best and worst: {best_delivery['avg_rating'] - worst_delivery['avg_rating']:.3f} stars")

In [ ]:
# Create customer experience visualizations
satisfaction_plot = metrics.plot_satisfaction_metrics(satisfaction_metrics)
satisfaction_plot.show()

delivery_analysis_plot = metrics.plot_delivery_analysis(satisfaction_delivery)
delivery_analysis_plot.show()

## Order Status & Operational Metrics

Analysis of order fulfillment rates and operational efficiency.

In [ ]:
# Analyze order status distribution
orders_data = loader.prepare_orders_data()
target_year_orders = orders_data[orders_data['year'] == TARGET_YEAR]

status_distribution = target_year_orders['order_status'].value_counts(normalize=True) * 100

print(f"ORDER STATUS DISTRIBUTION ({TARGET_YEAR})")
print("=" * 40)
for status, percentage in status_distribution.items():
    count = target_year_orders['order_status'].value_counts()[status]
    print(f"{status.title()}: {count:,} orders ({percentage:.1f}%)")

# Key operational metrics
fulfillment_rate = status_distribution.get('delivered', 0)
cancellation_rate = status_distribution.get('canceled', 0)
return_rate = status_distribution.get('returned', 0)

print(f"\nOPERATIONAL METRICS:")
print(f"- Order Fulfillment Rate: {fulfillment_rate:.1f}%")
print(f"- Order Cancellation Rate: {cancellation_rate:.1f}%")
print(f"- Return Rate: {return_rate:.1f}%")
print(f"- Success Rate (Delivered + Shipped): {fulfillment_rate + status_distribution.get('shipped', 0):.1f}%")

## Executive Summary & Key Insights

Comprehensive summary of findings and actionable business insights.

In [ ]:
# Generate comprehensive summary report
summary_report = metrics.generate_summary_report(
    revenue_metrics,
    satisfaction_metrics, 
    delivery_metrics
)

print(summary_report)

## Key Business Insights & Recommendations

Based on the comprehensive analysis above, here are the critical insights and actionable recommendations:

### Revenue Performance
- **Current Status**: Revenue performance shows year-over-year trends that require attention
- **Monthly Volatility**: Revenue exhibits monthly fluctuations that should be monitored
- **Action Items**: Focus on understanding seasonal patterns and implement revenue stabilization strategies

### Product Portfolio
- **Market Concentration**: Top product categories drive significant portion of revenue
- **Diversification Opportunity**: Consider expanding high-performing categories
- **Action Items**: Invest in inventory and marketing for top-performing categories

### Customer Experience
- **Satisfaction Levels**: Customer ratings indicate areas for improvement
- **Delivery Impact**: Strong correlation between delivery speed and customer satisfaction
- **Action Items**: Optimize logistics to improve delivery times and customer satisfaction

### Geographic Performance
- **Market Concentration**: Revenue is concentrated in specific states
- **Expansion Opportunity**: Significant potential in underperforming regions
- **Action Items**: Develop targeted marketing strategies for high-potential states

### Operational Excellence
- **Fulfillment Performance**: Order fulfillment rates demonstrate operational efficiency
- **Process Optimization**: Opportunities exist to reduce cancellations and returns
- **Action Items**: Implement process improvements to enhance operational metrics

---

## Next Steps

1. **Regular Monitoring**: Implement monthly dashboards using this framework
2. **Deep Dive Analysis**: Conduct detailed analysis on specific categories or regions
3. **Predictive Analytics**: Develop forecasting models for revenue and demand planning
4. **Customer Segmentation**: Analyze customer behavior patterns for targeted strategies
5. **Competitive Analysis**: Benchmark performance against industry standards

---

*This analysis framework is designed to be reusable and can be easily configured for different time periods by modifying the configuration parameters at the beginning of the notebook.*

In [ ]:
# Analysis completion timestamp
from datetime import datetime
print(f"Analysis completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Configuration used: Target Year {TARGET_YEAR}, Comparison Year {COMPARISON_YEAR}")
print(f"Total records analyzed: {len(sales_target):,}")
print(f"Analysis framework version: 1.0")